In [2]:
pip install datasketch

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 73 kB 14 kB/s eta 0:00:010
Note: you may need to restart the kernel to use updated packages.


In [19]:
# 对微博新闻进行相似句子Top-K查询
from datasketch import MinHash, MinHashLSH, MinHashLSHForest
from sklearn.feature_extraction.text import TfidfVectorizer
import jieba.posseg as pseg
import re

In [28]:
# 读取文件
f = open('weibos.txt', 'r', encoding='UTF-8')
text = f.read()
# 以句号，叹号，问号作为分隔，去掉\n换行符号
sentences = re.split('[。！？]', text.replace('\n', ''))

# 最后一行如果为空，则删除
if sentences[len(sentences)-1] == '':
    sentences.pop()
# print(sentences)
# print(text)

In [21]:
# 将item_text进行分词
def get_item_str(item_text):
    item_str = "" 
    item=(pseg.cut(item_text)) 
    for i in list(item):
        #去掉停用词
        if i.word not in list(stop):  
            item_str += i.word
            #tfidf_vectorizer.fit_transform的输入需要空格分隔的单词
            item_str += " "
    return item_str

In [22]:
# 对item_str创建MinHash
def get_minhash(item_str):
    temp = MinHash()
    for d in item_str:
        temp.update(d.encode('utf8'))
    return temp

In [23]:
# 设置停用词
stop = [line.strip().decode('utf-8') for line in open('stopword.txt').readlines()]
# 得到分词后的documents
documents = []
for item_text in sentences:
    # 将item_text进行分词
    item_str = get_item_str(item_text)
    documents.append(item_str)

In [24]:
# 创建LSH Forest及MinHash对象
minhash_list = []
forest = MinHashLSHForest()
for i in range(len(documents)):
    #得到train_documents[i]的MinHash
    temp = get_minhash(documents[i])
    minhash_list.append(temp)
    forest.add(i, temp)
# index所有key，以便可以进行检索
forest.index()

In [25]:
query = '里皮其实在宣布请辞同时已经去意已决'
# 将item_text进行分词
item_str = get_item_str(query)
# 得到item_str的MinHash
minhash_query = get_minhash(item_str)

# 查询forest中与m1相似的Top-K个邻居
print("与'{}'相似的Top3邻居:{}".format(query, result))
result = forest.query(minhash_query, 3)
for i in range(len(result)):
    print(result[i], minhash_query.jaccard(minhash_list[result[i]]), documents[result[i]].replace(' ', ''))

与'里皮其实在宣布请辞同时已经去意已决'相似的Top3邻居:[25, 4, 13]
25 0.1171875 ”中国足协：接受里皮辞职请求，将深刻反思看了个报道，马达洛尼说：“关于里皮的辞职，我事先也没有被告知，自己也不清楚发生了什么，也许是里皮头脑一热的决定
4 0.4453125 ​据了解，无论中国足协态度如何，里皮其实在宣布请辞同时已经去意已决
13 0.109375 当然，老头也是世界杯冠军教头，万一折在中国这里也没啥丢人的，毕竟里皮也折了嘛
